In [54]:
import cPickle as pickle
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from collections import Counter
import sklearn.cross_validation as cv
import random
import numpy as np
from copy import deepcopy
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [72]:
info = load_pickle("random.pkl")


In [17]:
def accuracy(svm, X, y):
    preds = svm.predict(X)
    print Counter(preds)
    wrong = 0
    dist = 0
    for (p,l) in zip(preds,y):
        if int(p) != int(l):
            wrong += 1
            dist += abs(l-p)/2.0
    print "correct predictions: %f%%"%((len(y)-wrong)/float(len(y)))
    print "average error: %f"%(dist/wrong)

In [50]:

# Input is a list of dictionarys mapping features to values (defined in pickle) 
def transform_to_XY(info):
    X = []
    Y = []
    feature_names = [i for i in info[0].keys()]
    total_vals = {}
    for x in feature_names:
        total_vals[x] = set()
    
    to_ignore = set(['name', 'rating', 'location', 'categories', 'url', 'parking', 'Best Nights','Good For'])
    
    for res in info:
        for k,v in res.items():
            if k not in to_ignore: # Some features are of list type or already floats, or not primitive types
                total_vals[k].add(v)
    
    #print total_vals
    feature_value_map = {}
    for k,v in total_vals.items():
        z = 0
        feature_value_map[k] = {}
        for value in v:
            feature_value_map[k][value] = z
            z += 1
            
    features = deepcopy(info)
    for b in features:
        #b["nones"] = sum([1 if b[f] == None else 0 for f in b])
        try:
            b["postal"] = int(b["location"].postal_code)
        except:
            b["postal"] =  0
        for f in to_ignore:
            b.pop(f, None)
        for f in b:
            if f not in to_ignore and f in feature_value_map and b[f] in feature_value_map[f]:
                if f not in ["review_count",'number_of_images']:
                    b[f] = feature_value_map[f][b[f]]
    features = [[d[f] for f in sorted(d.keys())] for d in features]
    return (features, [f["rating"]*2 for f in info])

In [57]:
clf = svm.SVC(kernel='rbf', tol = .001, probability = True)
clf.fit(X, y)  
accuracy(clf, Xp, yp)
print clf.score(Xp,yp)

Counter({8.0: 1308})
correct predictions: 0.342508%
average error: 0.805814
0.34250764526


In [27]:
clf2 = RandomForestClassifier(n_estimators=50)
clf2 = clf2.fit(X, y)
accuracy(clf2, Xp, yp)

Counter({8.0: 710, 7.0: 335, 9.0: 122, 6.0: 60, 10.0: 26, 5.0: 23, 4.0: 20, 2.0: 10, 3.0: 2})
correct predictions: 0.382263%
average error: 0.778465


In [60]:
clf3 = LogisticRegression()
clf3 = clf3.fit(X, y)
accuracy(clf3, Xp, yp)

Counter({8.0: 1304, 10.0: 4})
correct predictions: 0.342508%
average error: 0.805814


In [ ]:

print Counter(yp)

In [137]:
infotrain = filter(lambda x: x["review_count"] > 50, info)
#infotest = filter(lambda x: x["review_count"] > 50, info)
(X, y) = transform_to_XY(infotrain)
pca = PCA(n_components=20)
pca.fit(X)
X = pca.transform(X)
#X = new_info[:5000]
#Xp = new_info[5000:]
#yp = y[5000:]
#y = y[:5000]
print len(X),len(y)


2977 2977


In [81]:
c = Counter(y)
vals = [[x, c[x]/float(len(y))] for x in c]
for i in range(1,len(vals)):
    vals[i][1] += vals[i-1][1]
rands = []
print vals
for x in range(len(y)):
    r = random.random()
    i = 0
    while r > vals[i][1]:
        i += 1
    rands.append(vals[i][0])

wrong = 0
dist = 0
for (p,l) in zip(rands,y):
    if int(p) != l:
        wrong += 1
        dist += abs(l-p)/2.0
print "correct predictions: %f%%"%((len(y)-wrong)/float(len(y)))
print "average error: %f"%(dist/wrong)


[[2.0, 0.012048192771084338], [3.0, 0.0212428662016487], [4.0, 0.0523145212428662], [5.0, 0.10478757133798351], [6.0, 0.21781864299302472], [7.0, 0.4524413443246671], [8.0, 0.7883639822447686], [9.0, 0.9576727964489538], [10.0, 1.0]]
correct predictions: 0.336874%
average error: 0.920631


In [150]:
#averaging function from: http://scikit-learn.org/stable/modules/cross_validation.html

def cross_val(model, folds):
    (X, y) = transform_to_XY(infotrain)
    pca = PCA(n_components='mle')
    pca.fit(X)
    X = pca.transform(X)
    scores = cv.cross_val_score(model,X,y, cv = folds)
    print scores
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [151]:
cross_val(RandomForestClassifier(n_estimators=100),5)
#cross_val(svm.SVC(kernel='sigmoid', tol = .001, probability = True),5)


38
37
[ 0.45333333  0.43288591  0.4487395   0.55555556  0.53378378]
Accuracy: 0.48 (+/- 0.10)
